## 1. Importar Bibliotecas

In [5]:
import pandas as pd
import json
from collections import Counter
import requests
import csv

## 2. Coleta de Dados da API

In [6]:
url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjce/_search"
api_key = "APIKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw=="

payload = json.dumps({
    "size": 10000,
    "query": {"match": {"assuntos.codigo": "12487"}},
    "sort": [{"dataAjuizamento": {"order": "desc"}}]
})

headers = {
    'Authorization': api_key,
    'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)
dados_dict = response.json()
print(f"Total de processos encontrados: {dados_dict['hits']['total']['value']}")

# Salvando em Json
with open("data/dados_completos.json", "w", encoding="utf-8") as f:
    json.dump(dados_dict, f, indent=2, ensure_ascii=False)
print("Dados completos salvos em 'data/dados_completos.json'")

# Salvando em csv
processos = dados_dict["hits"]["hits"]

numeros = []
for processo in processos:
    numero_processo = processo["_source"]["numeroProcesso"]
    numeros.append(numero_processo)

Total de processos encontrados: 6046


FileNotFoundError: [Errno 2] No such file or directory: 'data/dados_completos.json'

## 3. Processamento dos Dados (Python Dataframe)

In [35]:
# Extrair dados dos processos
processos = []

for hit in dados_dict['hits']['hits']:
    processo = hit['_source']
    numero_processo = processo['numeroProcesso']
    grau = processo['grau']
    classe = processo['classe']['nome']
    assuntos = processo['assuntos']
    data_ajuizamento = processo['dataAjuizamento']
    ultima_atualizacao = processo['dataHoraUltimaAtualizacao']
    formato = processo['formato']['nome']
    codigo = processo['orgaoJulgador']['codigo']
    orgao_julgador = processo['orgaoJulgador']['nome']
    municipio = processo['orgaoJulgador']['codigoMunicipioIBGE']
    
    try:
        movimentos = processo['movimentos']
    except:
        movimentos = []
    
    processos.append([numero_processo, classe, data_ajuizamento, ultima_atualizacao, formato,
                     codigo, orgao_julgador, municipio, grau, assuntos, movimentos])

df = pd.DataFrame(processos, columns=['numero_processo', 'classe', 'data_ajuizamento', 'ultima_atualizacao',
                                      'formato', 'codigo', 'orgao_julgador', 'municipio', 'grau', 'assuntos', 'movimentos'])

print(f"Total de processos no DataFrame: {len(df)}")
df.sample(5)

Total de processos no DataFrame: 6046


,numero_processo,classe,data_ajuizamento,ultima_atualizacao,formato,codigo,orgao_julgador,municipio,grau,assuntos,movimentos
2479,02490315720238060001,Procedimento Comum Cível,20230725000000,2024-10-09T10:06:58.529Z,Eletrônico,8497,21ª VARA CIVEL DA COMARCA DE FORTALEZA,2304400,G1,"[{'codigo': 12487, 'nome': 'Fornecimento de me...","[{'complementosTabelados': [{'codigo': 18, 'va..."
4852,06220185520228060000,Agravo de Instrumento,20220210000000,2024-08-20T15:01:34.500Z,Eletrônico,89166,GADES - FRANCISCO LUCIDIO DE QUEIROZ JUNIOR,2304400,G2,"[{'codigo': 12487, 'nome': 'Fornecimento de me...","[{'complementosTabelados': [{'codigo': 2, 'val..."
5074,06363566820218060000,Agravo de Instrumento,20211104000000,2024-08-20T14:17:52.105Z,Eletrônico,84275,GADES - CARLOS AUGUSTO GOMES CORREIA,2304400,G2,"[{'codigo': 12487, 'nome': 'Fornecimento de me...","[{'complementosTabelados': [{'codigo': 4, 'val..."
108,30011924420258060115,Procedimento Comum Cível,20250711000000,2025-10-13T12:27:43.218Z,Eletrônico,82391,1ª VARA CIVEL DA COMARCA DE LIMOEIRO DO NORTE,2307601,G1,"[{'codigo': 12487, 'nome': 'Fornecimento de me...","[{'complementosTabelados': [{'codigo': 19, 'va..."
2003,02773096820238060001,Cumprimento de sentença,20231117000000,2025-09-10T20:33:09.973Z,Eletrônico,90332,NUCLEO DE JUSTIÇA 4.0 - CUMPRIMENTO DE SENTENÇ...,0,G1,"[{'codigo': 12416, 'nome': 'Tutela de Urgência...","[{'complementosTabelados': [{'codigo': 4, 'val..."


## 4. Análise de Decisões
#### Aqui podemos restringir os termos para "procedente/improcedente" ou expandir

In [36]:
# Extrair decisões dos movimentos
decisoes_por_processo = []
tipos_decisao_contagem = []

for idx, row in df.iterrows():
    numero = row['numero_processo']
    movimentos = row['movimentos']
    
    decisoes_encontradas = []
    
    if movimentos:
        for mov in movimentos:
            nome_mov = mov.get('nome', '')
            
            termos_decisao = [
                "Procedência",
                "Improcedência",
                "Improcedência do pedido e improcedência do pedido contraposto"
            ]
            
            # Identificar decisões
            if any(palavra in nome_mov for palavra in termos_decisao):
                decisoes_encontradas.append(nome_mov)
                tipos_decisao_contagem.append(nome_mov)
    
    if decisoes_encontradas:
        decisoes_por_processo.append({
            'numero_processo': numero,
            'decisoes': decisoes_encontradas
        })

print(f"Processos com decisões: {len(decisoes_por_processo)} de {len(df)}")
print(f"\nTipos de decisões encontradas:")
for tipo, count in Counter(tipos_decisao_contagem).most_common(10):
    print(f"  {tipo}: {count}")

Processos com decisões: 2212 de 6046

Tipos de decisões encontradas:
  Procedência: 1616
  Procedência em Parte: 305
  Improcedência: 259
  Procedência em parte do pedido e improcedência do pedido contraposto: 30
  Improcedência do pedido e improcedência do pedido contraposto: 14
  Procedência do pedido e improcedência do pedido contraposto: 5
  Procedência do Pedido - Reconhecimento pelo réu: 3
  Procedência em parte do pedido e procedência do pedido contraposto: 1
  Procedência do pedido e procedência do pedido contraposto: 1


In [37]:
# Criar DataFrame de decisões
decisoes_lista = []

for item in decisoes_por_processo:
    for decisao in item['decisoes']:
        decisoes_lista.append({
            'numero_processo': item['numero_processo'],
            'tipo_decisao': decisao
        })

df_decisoes = pd.DataFrame(decisoes_lista)
df_decisoes.head(10)

,numero_processo,tipo_decisao
0,30039390220258060071,Improcedência
1,02187331420258060001,Procedência
2,30465508320258060001,Procedência em Parte
3,30415587920258060001,Procedência em Parte
4,30048329320258060167,Procedência
5,02162085920258060001,Procedência
6,30355412720258060001,Procedência
7,30022788520258060071,Procedência
8,30006925120258060220,Improcedência
9,30303751420258060001,Procedência em Parte


## 5. Seleção Estratificada de Registros
#### Pegar todos os "Improcedência" (259 + 14) e 273 "Procedência" de forma distribuída

In [44]:
# Separar decisões por tipo
df_procedencia = df_decisoes[df_decisoes['tipo_decisao'] == 'Procedência'].copy()
df_improcedencia = df_decisoes[df_decisoes['tipo_decisao'] == 'Improcedência'].copy()
df_improcedencia_contraposto = df_decisoes[df_decisoes['tipo_decisao'] == 'Improcedência do pedido e improcedência do pedido contraposto'].copy()

# Registros de Improcedência (273 registros: 259 + 14)
df_improcedencias_todos = pd.concat([df_improcedencia, df_improcedencia_contraposto])

# Seleção estratificada de Procedência (273 registros)
procedencias = []
tamanho_bloco = 20
pulo = 60
posicao = 0
total_procedencia = len(df_procedencia)

while len(procedencias) < 361 and posicao < total_procedencia:
    # Pegar um bloco de 20 registros
    fim_bloco = posicao + tamanho_bloco
    # Colocar índices de i variando de posicao até fim_bloco na lista "procedencias"
    procedencias.extend(range(posicao, fim_bloco))

    # Pular 100 registros
    posicao = fim_bloco + pulo
    
# Garantir que temos exatamente 273 ao invés de 280
procedencias = procedencias[:361]

# Selecionar os registros de Procedência
df_procedencia_selecionada = df_procedencia.iloc[procedencias].copy()

# Combinar todos os registros selecionados
df_decisoes_balanceado = pd.concat([df_procedencia_selecionada, df_improcedencias_todos])

print(f"\nTotal de registros no dataset balanceado: {len(df_decisoes_balanceado)}")
print(f"  - Procedência: {len(df_procedencia_selecionada)}")
print(f"  - Improcedência: {len(df_improcedencia)}")
print(f"  - Improcedência do pedido e improcedência do pedido contraposto: {len(df_improcedencia_contraposto)}")

# Mostrar distribuição
df_decisoes_balanceado.head(10)


Total de registros no dataset balanceado: 634
  - Procedência: 361
  - Improcedência: 259
  - Improcedência do pedido e improcedência do pedido contraposto: 14


,numero_processo,tipo_decisao
1,02187331420258060001,Procedência
4,30048329320258060167,Procedência
5,02162085920258060001,Procedência
6,30355412720258060001,Procedência
7,30022788520258060071,Procedência
10,30292622520258060001,Procedência
14,30012724320258060071,Procedência
15,30193154420258060001,Procedência
16,30186884020258060001,Procedência
17,30182752720258060001,Procedência


## 6. Exportar Resultados

In [ ]:
# Salvar DataFrames em CSV
df.to_csv('data/processos_completo.csv', sep=',', header=True, index=False)
df_decisoes_balanceado.to_csv('data/decisoes_resumo.csv', sep=',', header=True, index=False)

# Salvar números de processos balanceados em CSV
numeros_balanceados = df_decisoes_balanceado['numero_processo'].unique()
with open('data/numeros_processos.csv', 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['numeroProcesso'])
    for num in numeros_balanceados:
        writer.writerow([num])

# Salvar dados completos em JSON
with open('data/processos_com_decisoes.json', 'w', encoding='utf-8') as f:
    json.dump(decisoes_por_processo, f, indent=2, ensure_ascii=False)

print("Arquivos exportados:")
print("  - data/processos_completo.csv")
print("  - data/decisoes_resumo.csv (546 registros balanceados)")
print(f"  - data/numeros_processos.csv ({len(numeros_balanceados)} processos únicos)")
print("  - data/processos_com_decisoes.json")

Arquivos exportados:
  - processos_completo.csv
  - decisoes_resumo.csv (546 registros balanceados)
  - numeros_processos.csv (628 processos únicos)
  - processos_com_decisoes.json


## 7. Próximos Passos: Coleta Completa de Dados

Após gerar os arquivos básicos acima, execute o pipeline completo de coleta de dados:

### Opção 1: Pipeline Completo (3 etapas)
```bash
python coletar_dados_ml.py
```

Isso executará:
1. **Scraping** - Coleta juiz e requerente do TJCE
2. **Inferência de Sexo** - Infere sexo dos nomes
3. **Geração de Features** - Gera dataset ML com 29 features

### Opção 2: Executar Etapas Individuais
```bash
# Apenas scraping
python coletar_dados_ml.py --etapa scraping

# Apenas inferência de sexo
python coletar_dados_ml.py --etapa inferir_sexo

# Apenas geração de features
python coletar_dados_ml.py --etapa features
```

### Arquivos Gerados
- `data/dados_processos_tjce.csv` - Dados do scraping
- `data/dados_processos_com_sexo.csv` - Dados + sexo inferido
- `data/dataset_ml_completo.csv` - Dataset final (29 features)